# Piramide de vacinação brasileira

Fontes:

1. população brasileira - [Projeção da população do Brasil e das Unidades da Federação - IBGE](https://www.ibge.gov.br/apps/populacao/projecao//index.html)
2. população vacinada - [Registros de Vacinação COVID19 - OpenDataSUS](https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8)

**ANTES DE RODAR O CÓDIGO**

- Baixe o ```.csv``` completo com os dados de vacinação no Brasil e salve o arquivo na mesma pasta aonde o jupyter notebook está.

- No local de date coloque a data de corte para até onde você irá pegar dados. Se deseja pegar dados de vacinação até o dia 01/06/2021, por exemplo, coloque a data como 02/06/2021.

- O ```.csv``` é grande demais para ser aberto sem estourar a RAM do meu computador (se o seu aguenta parabéns, você tem um ótimo computador). Portanto, este código faz um loop que abre ele em partes de 7 milhões de linhas (se quiser mudar isso, mude a variável ```rows_n``` para o valor que queira). Logo, antes de rodar o código, você precisa checar quantas pessoas já foram vacinadas até a data que deseja. Por exemplo, se já 140 milhões de pessoas já foram vacinadas até a data que você quer, o primeiro subloop deve ir até 140 milhões/7 milhões = 20

- Para checar quantas pessoas já foram vacinadas, recomendo acessar [ourworldindata.org/vaccination_brazil](ourworldindata.org/vaccination_brazil).

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as st
import os
import datetime
import time

from tqdm.notebook import tqdm

In [2]:
dates = ['2021-01-24', '2021-01-31', '2021-02-07', '2021-02-14', '2021-02-21',
         '2021-02-28', '2021-03-07', '2021-03-14', '2021-03-21', '2021-03-28',
         '2021-04-04', '2021-04-11', '2021-04-18', '2021-04-25', '2021-05-02',
         '2021-05-09', '2021-05-16', '2021-05-23', '2021-05-30', '2021-06-06',
         '2021-06-13', '2021-06-20', '2021-06-27', '2021-07-04', '2021-07-11',
         '2021-07-18', '2021-07-25', '2021-08-01', '2021-08-08', '2021-08-15',
         '2021-08-22', '2021-08-29', '2021-09-05', '2021-09-12', '2021-09-19',
         '2021-09-26']
dates_pd = pd.DataFrame({'dates': dates})
dates_pd['dates'] = pd.to_datetime(dates_pd['dates'], format = '%Y-%m-%d').dt.tz_localize(None)

In [3]:
pop_br = 212981000
cols = ['paciente_idade', 'vacina_dataaplicacao', 'vacina_descricao_dose',
        'paciente_enumsexobiologico']

man = [0.0356,0.0354,0.0357,0.0380,0.0411,0.0401,0.0403,0.0394,0.0358,0.0311,0.0285,0.0251,0.0205,
       0.0158,0.0113,0.0073,0.0045,0.0022,0.0013]
woman = [0.0340,0.0338,0.0342,0.0366,0.0402,0.0401,0.0410,0.0410,0.0379,0.0334,0.0311,0.0281,0.0238,
         0.0189,0.0142,0.0098,0.0067,0.0037,0.0026]

br = pd.DataFrame({'Age': ['0-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44',
                           '45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85-89','90+'],
                   'Male': np.array(man)*pop_br,
                   'Female': np.array(woman)*(-pop_br)})
AgeClass = ['90+','85-89','80-84','75-79','70-74','65-69','60-64','55-59','50-54','45-49','40-44',
            '35-39','30-34','25-29','20-24','15-19','10-14','5-9','0-4']

In [4]:
rows_n = 1e6  # adjust it according to RAM capacity - My wsl can't handle more than 3e6 :(
size = 34
csv_vacinacao_brazil = '/mnt/d/Downloads/covid_vacinacao_brazil.csv'

In [5]:
df_iter = pd.read_csv(csv_vacinacao_brazil, chunksize=rows_n, error_bad_lines=False, delimiter = ';')

vac_dict = {}


for vac in tqdm(df_iter, total=220):
    # makes the vaccination date a datetime obj
    vac['vacina_dataaplicacao'] = pd.to_datetime(vac.vacina_dataaplicacao, 
                                                     format='%Y-%m-%d').dt.tz_localize(None)
    male_first = []
    male_second = []

    female_first = []
    female_second = []

    for date in dates:
        # separa os grupos para as n doses
        vacina_1_dose = vac.loc[(vac['vacina_descricao_dose'] == '1ª\xa0Dose') &
                                (vac['vacina_dataaplicacao'] < datetime.datetime.fromisoformat(date))
                               ]
        vacina_2_dose = vac.loc[(vac['vacina_descricao_dose'] == '2ª\xa0Dose') &
                                (vac['vacina_dataaplicacao'] < datetime.datetime.fromisoformat(date))
                               ]
        vacina_dose_unica = vac.loc[(vac['vacina_descricao_dose'] == 'Dose\xa0') &
                                    (vac['vacina_dataaplicacao'] < datetime.datetime.fromisoformat(date))
                                   ]

        # primeira dose grupos de 0 a 90+ anos separado em grupos de 4 em 4 anos
        g_first_dose = [vacina_1_dose.loc[(vacina_1_dose['paciente_idade'] >= j*5) & 
                               (vacina_1_dose['paciente_idade'] <= j*5 + 4)] 
                        if j != 18 
                        else vacina_1_dose.loc[vacina_1_dose['paciente_idade'] >= j*5]
                        for j in range(19) ]

        # segunda dose grupos de 0 a 90+ anos separado em grupos de 4 em 4 anos
        g_second_dose = [vacina_2_dose.loc[(vacina_2_dose['paciente_idade'] >= j*5) & 
                                (vacina_2_dose['paciente_idade'] <= j*5 + 4)]
                         if j != 18 
                         else vacina_2_dose.loc[vacina_2_dose['paciente_idade'] >= j*5]
                         for j in range(19)]

        # terceira dose grupos de 0 a 90+ anos separado em grupos de 4 em 4 anos
        g_single_dose = [vacina_dose_unica.loc[(vacina_dose_unica['paciente_idade'] >= j*5) &
                                    (vacina_dose_unica['paciente_idade'] <= j*5 + 4)]
                         if j != 18 
                         else vacina_dose_unica.loc[vacina_dose_unica['paciente_idade'] >= j*5]
                         for j in range(19)]

        # faz a contagem dos pacientes de sexo masculino para a idade especificada
        # para 1a, 2a e dose unica
        male_first = [len(x.loc[x['paciente_enumsexobiologico'] == 'M']) for x in g_first_dose]
        male_second = [len(x.loc[x['paciente_enumsexobiologico'] == 'M']) + 
                       len(x2.loc[x2['paciente_enumsexobiologico'] == 'M']) 
                       for x, x2 in zip(g_second_dose, g_single_dose)]

        female_first = [len(x.loc[x['paciente_enumsexobiologico'] == 'F']) for x in g_first_dose]
        female_second = [len(x.loc[x['paciente_enumsexobiologico'] == 'F']) + 
                         len(x2.loc[x2['paciente_enumsexobiologico'] == 'F']) 
                         for x, x2 in zip(g_second_dose, g_single_dose)]

        male_first = np.array(male_first)
        male_second = np.array(male_second)

        female_first = np.array(female_first)
        female_second = np.array(female_second)

        vac_dict[date] = {'f_first': vac_dict[date]['f_first'] + female_first 
                                                                      if date in vac_dict else female_first, 
                                  'f_second': vac_dict[date]['f_second'] + female_second 
                                                                      if date in vac_dict else female_second, 
                                  'm_first': vac_dict[date]['m_first'] + male_first 
                                                                      if date in vac_dict else male_first, 
                                  'm_second': vac_dict[date]['m_second'] + male_second 
                                                                      if date in vac_dict else male_second}


In [6]:
import json

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

json.dump(vac_dict, open('vac_dict.json', 'w'), cls=NumpyEncoder)

In [9]:
# Construção dos gráficos

for date in vac_dict.keys():
    df = pd.DataFrame({'Age': ['0-4','5-9','10-14','15-19','20-24','25-29',
                               '30-34', '35-39','40-44','45-49','50-54',
                               '55-59','60-64','65-69','70-74','75-79',
                               '80-84','85-89','90+'],
                       'Male_1dose': vac_dict[date]['m_first'],
                       'Male_2dose': vac_dict[date]['m_second'],
                       'Female_1dose': -np.array(vac_dict[date]['f_first']),
                       'Female_2dose': -np.array(vac_dict[date]['f_second'])})
    plt.figure(figsize=(10,8))
    bar_plot = sns.barplot(x='Male', 
                           y='Age', 
                           data=br, 
                           order=AgeClass, 
                           color = 'blue', 
                           alpha = 0.4, 
                           label = 'Homens população')
    bar_plot = sns.barplot(x='Female', 
                           y='Age', 
                           data=br, 
                           order=AgeClass, 
                           color = 'red', 
                           alpha = 0.4, 
                           label = 'Mulheres população')
    bar_plot = sns.barplot(x='Male_1dose', 
                           y='Age', 
                           data=df, 
                           order=AgeClass, 
                           color = 'blue', 
                           alpha = 0.7, 
                           label = 'Homens vacinados 1ª dose')
    bar_plot = sns.barplot(x='Female_1dose', y='Age', 
                           data=df, 
                           order=AgeClass, 
                           color = 'red', 
                           alpha = 0.7, 
                           label = 'Mulheres vacinadas 1ª dose')

    bar_plot = sns.barplot(x='Male_2dose', 
                           y='Age', 
                           data=df, 
                           order=AgeClass, 
                           color = 'darkblue', 
                           label = 'Homens vacinados 2ª dose')
    bar_plot = sns.barplot(x='Female_2dose', 
                           y='Age', 
                           data=df, 
                           order=AgeClass, 
                           color = 'darkred', 
                           label = 'Mulheres vacinados 2ª dose')

    if not os.path.exists('./results'):
        os.mkdir('./results')

    plt.ylabel('Grupo Etário', fontsize = 14)
    plt.xlabel('População (milhões)', fontsize = 14)
    plt.title(f'Pirâmide de vacinação ({date})', fontsize = 18)
    plt.xticks([-10e6, -7.5e6, -5e6, -2.5e6, 0, 2.5e6, 5e6, 7.5e6, 10e6],
            labels = ['10','7.5','5','2.5','0','2.5','5','7.5','10'])
    plt.legend(title = 'Populações', bbox_to_anchor = (1.01,1), fontsize = 11)
    plt.savefig(f'./results/vaccination_pyramid_{date}.png', bbox_inches = 'tight', dpi = 300)
    plt.close()

In [10]:
from glob import glob
import imageio

# Build GIF
print('Creating gif\n')
list_images = sorted(glob('./results/*.png'))
with imageio.get_writer(f'vacinacao_brasil_2021-09-26.gif', mode='I', duration=0.2, subrectangles=True) as writer:
    for ix, filename in enumerate(list_images):
        image = imageio.imread(filename)
        writer.append_data(image)
        if ix + 1 == len(list_images):
            for i in range(10): writer.append_data(image)
print('Gif saved\n')

Creating gif

Gif saved

